In [1]:
from AlgoritmoGenetico import GeneticAlgorithm
import os
import pygame

pygame 2.6.1 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
# Parámetros del algoritmo genético
population_size = 30
mutation_rate = 0.01
num_generations = 30
input_size = 11  # Tamaño del vector de estado
output_size = 4  # Cuatro acciones posibles: izquierda, derecha, disparar, nada

os.environ["SDL_VIDEODRIVER"] = "dummy"  # Configurar el driver de video en "dummy"

In [ ]:
pygame.init()  # Asegúrate de inicializar Pygame
pygame.font.init()

gen_algo = GeneticAlgorithm(population_size, mutation_rate, num_generations, input_size, output_size)

print("Entrenando la red neuronal mediante algoritmo genético...")
trained_model = gen_algo.evolve()  # Este será el mejor modelo entrenado

trained_model.save("resultado_1")

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Entrenando la red neuronal mediante algoritmo genético...
Inciando generacion 0
Agente 0 Iniciado. Level: 1
Agente 1 Iniciado. Level: 1
Agente 2 Iniciado. Level: 1
Agente 3 Iniciado. Level: 1
Agente 4 Iniciado. Level: 1
Agente 5 Iniciado. Level: 1
Agente 6 Iniciado. Level: 1
Agente 7 Iniciado. Level: 1
Agente 8 Iniciado. Level: 1
Agente 9 Iniciado. Level: 1
Agente 10 Iniciado. Level: 1
Agente 11 Iniciado. Level: 1
Agente 12 Iniciado. Level: 1
Agente 13 Iniciado. Level: 1
Agente 14 Iniciado. Level: 1
Agente 15 Iniciado. Level: 1
Agente 16 Iniciado. Level: 1
Agente 17 Iniciado. Level: 1
Agente 18 Iniciado. Level: 1
Agente 19 Iniciado. Level: 1
Agente 20 Iniciado. Level: 1
Agente 21 Iniciado. Level: 1
Agente 22 Iniciado. Level: 1
Agente 23 Iniciado. Level: 1
Agente 24 Iniciado. Level: 1
Agente 25 Iniciado. Level: 1
Agente 26 Iniciado. Level: 1
Agente 27 Iniciado. Level: 1
Agente 28 Iniciado. Level: 1
Agente 29 Iniciado. Level: 1
[0.25 0.25 0.25 0.25]
[0.25 0.25 0.25 0.25]
[0.25 0.25 0.25 